In [4]:
using Finch
using SparseArrays
using Metatheory
using Metatheory.EGraphs
using TermInterface
using PrettyPrinting
include("../Source/Spartan.jl")

spartan (generic function with 1 method)

In [6]:
is = Set(["i"])
ls = Set(["l"])
a_N = 100000
a_indices = Set(["i", "j"])
a_dimSizes = Dict("i" => 1000, "j"=>1000)
a = InputTensor("A", TensorStats(a_indices, a_dimSizes, a_N, 0.0))
A_fiber = dropdefaults(copyto!(@fiber(sl(sl(e(0.0)))), sparse(rand(1:a_dimSizes["i"], a_N), rand(1:a_dimSizes["j"], a_N), ones(a_N),  a_dimSizes["i"], a_dimSizes["j"]))) # remove zeros
b_indices = Set(["j", "l"])
b_dimSizes = Dict("j" => 1000, "l" => 1000)
b_N = 100000
b = InputTensor("B", TensorStats(b_indices, b_dimSizes, b_N, 0.0))
B_fiber = dropdefaults(copyto!(@fiber(sl(sl(e(0.0)))), sparse(rand(1:b_dimSizes["j"], b_N),rand(1:b_dimSizes["l"], b_N), ones(b_N),b_dimSizes["j"],  b_dimSizes["l"])))
input_tensor_dict = Dict("A" => A_fiber, "B" => B_fiber)

# Sum_i Sum_l B[j,l]*A[i,j]*A[i,j]
output_tensor_1 = spartan(:(ReduceDim($+, $is, ReduceDim($+, $ls, MapJoin($*, $b, MapJoin($*, $a, $a))))), input_tensor_dict, verbose=true, optimize=false)
#println(output_tensor)

# (Sum_l B[j,l])*(Sum_i A[i,j] * A[i,j])
output_tensor_2 = spartan(:(ReduceDim($+, $is, ReduceDim($+, $ls, MapJoin($*, $b, MapJoin($*, $a, $a))))), input_tensor_dict, verbose=true, optimize=true)
#println(output_tensor)

println("Is Correct?: ", output_tensor_1 == output_tensor_2)

(Optimized) Expression: 
quote
    ReduceDim(
        +,
        Set(["i"]),
        ReduceDim(
            +,
            Set(["l"]),
            MapJoin(
                *,
                InputTensor("B", TensorStats(Set(["j", "l"]), Dict("j" => 1000, "l" => 1000), 100000.0, 0.0), nothing),
                MapJoin(
                    *,
                    InputTensor("A", TensorStats(Set(["j", "i"]), Dict("j" => 1000, "i" => 1000), 100000.0, 0.0), nothing),
                    InputTensor("A", TensorStats(Set(["j", "i"]), Dict("j" => 1000, "i" => 1000), 100000.0, 0.0), nothing)))))
end
1
2
3
4
4
Kernel: OperatorExpr(*, t_custom_op, InputTensorKernel[InputTensorKernel("A", TensorStats(Set{String}(), Dict{String, Int64}(), 0.0, 0)), InputTensorKernel("B", TensorStats(Set{String}(), Dict{String, Int64}(), 0.0, 0))])
Expected Output Tensor Size: 10000.000000000002
Output Tensor Size: 95129
3
Kernel: OperatorExpr(*, t_custom_op, InputTensorKernel[InputTensorKernel("A", TensorStats(Set{

In [5]:
is = Set(["i"])
js = Set(["j"])
N = 1000
x_N = 1000
x_indices = Set(["i", "j"])
x_dimSizes = Dict("i" => N, "j"=>N)
X = InputTensor("X", TensorStats(x_indices, x_dimSizes, x_N, 0.0))
X_fiber = dropdefaults(copyto!(@fiber(sl(sl(e(0.0)))), sparse(rand(1:x_dimSizes["j"], x_N), rand(1:x_dimSizes["i"], x_N), ones(x_N), x_dimSizes["j"], x_dimSizes["i"]))) # remove zeros
u_N = N
u_indices = Set(["i"])
u_dimSizes = Dict("i" => N)
u = InputTensor("u", TensorStats(u_indices, u_dimSizes, u_dimSizes["i"], 0.0))
u_fiber = dropdefaults(copyto!(@fiber(sh{1}(e(0.0))), ones(u_N))) 
v_N = N
v_indices = Set(["j"])
v_dimSizes = Dict("j" => N)
v = InputTensor("v", TensorStats(v_indices, v_dimSizes, v_dimSizes["j"], 0.0))
v_fiber = dropdefaults(copyto!(@fiber(sh{1}(e(0.0))), ones(v_N)))
input_tensor_dict = Dict("X" => X_fiber, "u" => u_fiber, "v" => v_fiber)

# Sum_i Sum_l B[j,l]*A[i,j]*A[i,j]
output_tensor_1 = spartan(:(ReduceDim($+, $is, ReduceDim($+, $js, MapJoin($^, MapJoin($+, $X, MapJoin($*, $u, $v)), 2)))), input_tensor_dict, verbose=true, optimize=false)

# (Sum_l B[j,l])*(Sum_i A[i,j] * A[i,j])
output_tensor_2 = spartan(:(ReduceDim($+, $is, ReduceDim($+, $js, MapJoin($^, MapJoin($+, $X, MapJoin($*, $u, $v)), 2)))), input_tensor_dict, verbose=true, optimize=true)

println("Is Correct?: ", output_tensor_1 == output_tensor_2)

(Optimized) Expression: 
quote
    ReduceDim(
        +,
        Set(["i"]),
        ReduceDim(
            +,
            Set(["j"]),
            MapJoin(
                ^,
                MapJoin(
                    +,
                    InputTensor("X", TensorStats(Set(["j", "i"]), Dict("j" => 1000, "i" => 1000), 1000.0, 0.0), nothing),
                    MapJoin(
                        *,
                        InputTensor("u", TensorStats(Set(["i"]), Dict("i" => 1000), 1000.0, 0.0), nothing),
                        InputTensor("v", TensorStats(Set(["j"]), Dict("j" => 1000), 1000.0, 0.0), nothing))),
                2)))
end
1
2
3
4
5
5
Kernel: OperatorExpr(*, t_custom_op, InputTensorKernel[InputTensorKernel("A", TensorStats(Set{String}(), Dict{String, Int64}(), 0.0, 0)), InputTensorKernel("B", TensorStats(Set{String}(), Dict{String, Int64}(), 0.0, 0))])
Expected Output Tensor Size: 1.0e6
Output Tensor Size: 1000000
4
Kernel: OperatorExpr(+, t_custom_op, InputTensorKernel[In

1.003e6

In [6]:
using Finch
t = @fiber(sl(sl(e(0.0))))
Finch.@finch_program_instance (t .= 0; @loop j l i t[l, j] += A_fiber[i,j] * B_fiber[j,l])

sequence_instance(declare_instance(variable_instance(:t, t), literal_instance(0)), loop_instance(index_instance(:j), loop_instance(index_instance(:l), loop_instance(index_instance(:i), assign_instance(access_instance(variable_instance(:t, t), updater_instance(create_instance()), index_instance(:l), index_instance(:j)), variable_instance(:+, +), call_instance(variable_instance(:*, *), access_instance(variable_instance(:A_fiber, A_fiber), reader_instance(), index_instance(:i), index_instance(:j)), access_instance(variable_instance(:B_fiber, B_fiber), reader_instance(), index_instance(:j), index_instance(:l))))))))


In [7]:
C = @fiber(sl(sl(e(0.0))))
prgm = Finch.@finch_program_instance (C .= 0; @loop i j C[j, i] = A_fiber[j, i] * B_fiber[j, i])


sequence_instance(declare_instance(variable_instance(:C, C), literal_instance(0)), loop_instance(index_instance(:i), loop_instance(index_instance(:j), assign_instance(access_instance(variable_instance(:C, C), updater_instance(create_instance()), index_instance(:j), index_instance(:i)), literal_instance(Finch.FinchNotation.InitWriter{0.0}()), call_instance(variable_instance(:*, *), access_instance(variable_instance(:A_fiber, A_fiber), reader_instance(), index_instance(:j), index_instance(:i)), access_instance(variable_instance(:B_fiber, B_fiber), reader_instance(), index_instance(:j), index_instance(:i)))))))


In [3]:
N=10000
A_fiber = dropdefaults(copyto!(@fiber(sl(sl(e(0.0)))), sparse(rand(1:a_dimSizes["j"], N), rand(1:a_dimSizes["i"], N), ones(N), a_dimSizes["j"], a_dimSizes["i"]))) # remove zeros
D = @fiber(sl(sl(sl(e(0.0)))))
@finch (D.=0; @loop l j i D[i,j,l] = A_fiber[i,j] * A_fiber[j,l])
countstored(D)

LoadError: UndefVarError: a_dimSizes not defined

In [32]:
C = @fiber(sl(sl(sl(e(0.0)))))
@finch (C .= 0; @loop  l i j C[i, l, j] = D[i,j,l])

(C = Fiber(SparseList{Int64, Int64}(SparseList{Int64, Int64}(SparseList{Int64, Int64}(Element{0.0, Float64}([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]), 10000, [1, 3, 8, 12, 16, 19, 20, 21, 25, 29  …  9861, 9862, 9864, 9866, 9868, 9869, 9870, 9871, 9872, 9873], [419, 6391, 2441, 4892, 6866, 6954, 9064, 1897, 4702, 6455  …  9516, 8842, 9516, 7047, 9014, 5601, 5601, 5601, 7622, 7622]), 10000, [1, 2, 3, 5, 6, 7, 8, 11, 12, 15  …  6232, 6234, 6235, 6236, 6238, 6239, 6241, 6242, 6245, 6247], [5020, 3375, 1821, 5941, 8359, 2837, 9311, 3002, 5799, 8907  …  9465, 2800, 2383, 6452, 1180, 1514, 3318, 4554, 571, 6510]), 10000, [1, 3973], [2, 4, 6, 8, 9, 13, 15, 17, 19, 25  …  9969, 9970, 9974, 9975, 9982, 9983, 9985, 9986, 9998, 10000])),)

In [35]:
C = @fiber(sl(sl(e(0.0))))
@finch (C .= 0; @loop  j i l C[i,j] += D[i,j,l])

In [ ]:
is = Set(["i"])
ks = Set(["k"])
a_indices = Set(["i", "j"])
a_dimSizes = Dict("i" => 1000, "j"=>100)
a = InputTensor(TensorStats(a_indices, a_dimSizes, 100000, 0.0), "A")
b_indices = Set(["j", "k"])
b_dimSizes = Dict("j" => 100, "k"=>1000)
b = InputTensor(TensorStats(b_indices, b_dimSizes, 100000, 0.0), "B")
c_indices = Set(["j", "l"])
c_dimSizes = Dict("j" => 100, "l" => 100)
c = InputTensor(TensorStats(c_indices, c_dimSizes, 10000, 0.0), "C")

# Sum_i Sum_k C[j,l] + B[j,k] * A[i,j]
g= EGraph(:(reduceDim($+, $is, reduceDim($+, $ks, MapJoin($+, $c, MapJoin($*, $b, $a))))))
analyze!(g, :TensorStatsAnalysis)
params = SaturationParams(timeout=10, eclasslimit=4000)
report = saturate!(g, basic_rewrites, params);

# Results In:(Sum_k,i C[j,l]) + (Sum_k B[j,k]) * (Sum_i A[i,j])
pprintln(extract!(g, simple_cardinality_cost_function))
print(report)
getdata(g[g.root], :TensorStatsAnalysis)

In [ ]:
is = Set(["i"])
js = Set(["j"])
ks = Set(["k"])
x_indices = Set(["i", "j"])
x_dimSizes = Dict("i" => 1000, "j"=>1000)
x = InputTensor(TensorStats(x_indices, x_dimSizes, 100, 0.0), "X")
u_indices = Set(["i", "k"])
u_dimSizes = Dict("i" => 1000, "k"=>10)
u = InputTensor(TensorStats(u_indices, u_dimSizes, 10000, 0.0), "U")
v_indices = Set(["k", "j"])
v_dimSizes = Dict("k" => 10, "j" => 1000)
v = InputTensor(TensorStats(v_indices, v_dimSizes, 10000, 0.0), "V")

# Sum_i Sum_k C[j,l] + B[j,k] * A[i,j]
g= EGraph(:(reduceDim($+, $is, reduceDim($+, $js, mapScalar($^, MapJoin($+, $x, reduceDim($+, $ks, MapJoin($*, $u, $v))), $2)))))

analyze!(g, :TensorStatsAnalysis)
params = SaturationParams(timeout=100, eclasslimit=4000)
report = saturate!(g, basic_rewrites, params);

# Results In:(Sum_k,i C[j,l]) + (Sum_i B[j,k]) * (Sum_k A[i,j])
pprintln(extract!(g, simple_cardinality_cost_function))
print(report)
getdata(g[g.root], :TensorStatsAnalysis)

In [11]:
is = Set(["i"])
js = Set(["j"])
ks = Set(["k"])
tensors = []
for i in 1:10
    indices = Set(["i_" *  string(i), "i_" * string(i+1)])
    dimSizes = Dict()
    for index in indices
        dimSizes[index] = 100
    end
    push!(tensors, InputTensor("A_" * string(i), TensorStats(indices, dimSizes, 10000, 0.0)))
end


# Optimization w/cross products
#g= EGraph(:(MapJoin($*, $(tensors[1]), MapJoin($*, $(tensors[2]), MapJoin($*, $(tensors[3]), MapJoin($*, $(tensors[4]), MapJoin($*, $(tensors[5]), MapJoin($*, $(tensors[6]), MapJoin($*, $(tensors[7]), MapJoin($*, $(tensors[8]), $(tensors[9])))))))))))
g= EGraph(:(MapJoin($*, $(tensors[1]), MapJoin($*, $(tensors[2]), MapJoin($*, $(tensors[3]), MapJoin($*, $(tensors[4]), MapJoin($*, $(tensors[5]), MapJoin($*, $(tensors[6]), MapJoin($*, $(tensors[7]), $(tensors[8]))))))))))
analyze!(g, :TensorStatsAnalysis)
params = SaturationParams(timeout=10, eclasslimit=100000)
report = saturate!(g, basic_rewrites, params);

pprintln(extract!(g, simple_cardinality_cost_function))
print(report)
getdata(g[g.root], :TensorStatsAnalysis)



# Optimization w/out cross-products
#g= EGraph(:(MapJoin($*, $(tensors[1]), MapJoin($*, $(tensors[2]), MapJoin($*, $(tensors[3]), MapJoin($*, $(tensors[4]), MapJoin($*, $(tensors[5]), MapJoin($*, $(tensors[6]), MapJoin($*, $(tensors[7]), MapJoin($*, $(tensors[8]), $(tensors[9])))))))))))
g= EGraph(:(MapJoin($*, $(tensors[1]), MapJoin($*, $(tensors[2]), MapJoin($*, $(tensors[3]), MapJoin($*, $(tensors[4]), MapJoin($*, $(tensors[5]), MapJoin($*, $(tensors[6]), MapJoin($*, $(tensors[7]), $(tensors[8]))))))))))
analyze!(g, :TensorStatsAnalysis)
params = SaturationParams(timeout=10, eclasslimit=100000)
report = saturate!(g, basic_rewrites_2, params);

pprintln(extract!(g, simple_cardinality_cost_function))
print(report)
getdata(g[g.root], :TensorStatsAnalysis)

LoadError: AssertionError: isempty(MERGES_BUF[])

63708